# Running Monte Carlo Transport Independently

This tutorial demonstrates how to run the Monte Carlo transport loop directly using `Simulation.from_config` without running full TARDIS iterations. This approach gives you direct control over the Monte Carlo transport process.

In [ ]:
from numba import config as nconfig

from tardis.transport.montecarlo.packets.trackers.tracker_last_interaction_util import generate_tracker_last_interaction_list, trackers_last_interaction_to_df

nconfig.DISABLE_JIT = False

from pathlib import Path

import astropy.units as u

from tardis.io.atom_data import AtomData
from tardis.io.configuration.config_reader import Configuration
from tardis.simulation import Simulation
from tardis.transport.montecarlo.estimators.radfield_mc_estimators import (
    initialize_estimator_statistics,
)
from tardis.transport.montecarlo.montecarlo_main_loop import (
    montecarlo_main_loop,
)
from tardis.transport.montecarlo.packet_source.black_body import (
    BlackBodySimpleSource,
)
from tardis.transport.montecarlo.packets.trackers import (
    generate_tracker_full_list,
    tracker_full_df2tracker_last_interaction_df,
    trackers_full_to_df,
)
# Import TrackerFullSolver and TrackerFullState from tardis.transport.montecarlo.packets.trackers.tracker_full_solver
from tardis.transport.montecarlo.packets.trackers.tracker_full_solver import TrackerFullSolver, TrackerFullState

In [ ]:
# Disable JIT compilation for debugging (WARNING: This breaks TARDIS plasma assembly)
# os.environ["NUMBA_DISABLE_JIT"] = "1"

# Alternative: Use selective debugging with numba.set_num_threads(1) for better stack traces

CONFIG_FILE_NAME = "tardis_example.yml"
NUMBER_OF_PACKETS = 10000
NUMBER_OF_VPACKETS = 0  # Set to 0 to disable virtual packets
ITERATION_NUMBER = 1
SHOW_PROGRESS_BARS = True
TOTAL_ITERATIONS = 1
ENABLE_RPACKET_TRACKING = (
    False  # True: full tracking, False: last interaction only
)

In [ ]:
# Setup simulation state from config
config_file = Path(CONFIG_FILE_NAME)
if not config_file.exists():
    raise FileNotFoundError(f"Configuration file {CONFIG_FILE_NAME} not found")

config = Configuration.from_yaml(str(config_file))
atom_data = AtomData.from_hdf("kurucz_cd23_chianti_H_He_latest.h5")
sim = Simulation.from_config(config, atom_data=atom_data)

In [ ]:
# Initialize opacity and macro atom states manually
sim.opacity_state = sim.opacity.legacy_solve(sim.plasma)

if sim.macro_atom is not None:
    sim.macro_atom_state = sim.macro_atom.solve(
        sim.plasma.j_blues,
        sim.opacity_state.beta_sobolev,
        sim.plasma.stimulated_emission_factor,
    )
else:
    sim.macro_atom_state = None

In [ ]:
# Extract states from simulation
geometry_state = sim.simulation_state.geometry
opacity_state = sim.opacity_state
montecarlo_configuration = sim.transport.montecarlo_configuration
time_explosion = sim.simulation_state.time_explosion.to(u.s).value
spectrum_frequency_grid = sim.transport.spectrum_frequency_grid.to(u.Hz).value

# Create our own independent packet source instead of reusing sim.transport.packet_source
packet_source = BlackBodySimpleSource(
    radius=geometry_state.r_inner_active[0],
    temperature=sim.simulation_state.t_inner,
    base_seed=23111963,  # Use inner temperature from simulation
)

# Initialize estimators
tau_sobolev_shape = opacity_state.tau_sobolev.shape
gamma_shape = (0, geometry_state.no_of_shells)
estimators = initialize_estimator_statistics(tau_sobolev_shape, gamma_shape)

# Convert to numba-compatible versions
geometry_state_numba = geometry_state.to_numba()
line_interaction_type = montecarlo_configuration.LINE_INTERACTION_TYPE
opacity_state_numba = opacity_state.to_numba(
    sim.macro_atom_state, line_interaction_type
)

## Creating Independent Packet Source

Instead of reusing the packet source from the simulation (`sim.transport.packet_source`), we create our own independent `BlackBodySimpleSource`. This gives us full control over the packet generation parameters and ensures our Monte Carlo run is completely independent of the simulation's transport state.

In [ ]:
ENABLE_RPACKET_TRACKING = False  # Test last interaction tracker

# Initialize TrackerFullSolver and state (not connected to MC loop)
tracker_solver = TrackerFullSolver(initial_length=1000, extend_factor=2)
tracker_state = tracker_solver.create_tracker_state(num_packets=100)

# Create packet collection using our independent packet source
seed_offset = 0
packet_collection = packet_source.create_packets(NUMBER_OF_PACKETS, seed_offset)

# Setup packet tracking
if ENABLE_RPACKET_TRACKING:
    rpacket_trackers = generate_tracker_full_list(
        NUMBER_OF_PACKETS,
        montecarlo_configuration.INITIAL_TRACKING_ARRAY_LENGTH,
    )
    rpacket_tracker_collection = None
else:
    # Initialize the last interaction tracker collection
    # Generate individual trackers for the main loop
    rpacket_trackers = generate_tracker_last_interaction_list(NUMBER_OF_PACKETS)

# Run the Monte Carlo main loop
v_packets_energy_hist, vpacket_tracker = montecarlo_main_loop(
    packet_collection,
    geometry_state_numba,
    time_explosion,
    opacity_state_numba,
    montecarlo_configuration,
    estimators,
    spectrum_frequency_grid,
    rpacket_trackers,
    NUMBER_OF_VPACKETS,
    SHOW_PROGRESS_BARS,
)

In [ ]:
%%timeit
v_packets_energy_hist, vpacket_tracker = montecarlo_main_loop(
    packet_collection,
    geometry_state_numba,
    time_explosion,
    opacity_state_numba,
    montecarlo_configuration,
    estimators,
    spectrum_frequency_grid,
    rpacket_trackers,
    NUMBER_OF_VPACKETS,
    SHOW_PROGRESS_BARS,
)

In [ ]:
# Create DataFrame from tracker data
if ENABLE_RPACKET_TRACKING:
    # Full tracking: convert from rpacket_trackers list
    # Create event dataframe (all events including boundary crossings)
    tracker_df = trackers_full_to_df(rpacket_trackers)

    # Create last interaction dataframe from full tracking
    last_tracker_df = tracker_full_df2tracker_last_interaction_df(tracker_df)

else:
    # Last interaction tracking: convert from rpacket_trackers list
    last_tracker_df = trackers_last_interaction_to_df(rpacket_trackers)

In [ ]:
tracker_df.loc[6]

In [ ]:
last_tracker_df

In [ ]:
last_tracker_df

In [ ]:
last_tracker_df.columns

In [ ]:
last_tracker_df_create = last_tracker_df.copy()

In [ ]:
last_tracker_df.loc[9]

In [ ]:
last_tracker_df.loc[9]